# Originality Algo

## Algorithm to Automate Originality Scoring

### Import Packages

In [2]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from functools import reduce
import openpyxl
import xlsxwriter

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.util import ngrams
from nltk import ngrams, FreqDist
from nltk.lm import NgramCounter
import string
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

from spacy.lang.en.stop_words import STOP_WORDS

from collections import Counter
import itertools

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

import matplotlib.pyplot as plt
from yellowbrick.cluster import KElbowVisualizer

from nltk.cluster.kmeans import KMeansClusterer

### Put Data from Excel Sheet into Dataframes

In [3]:
# individual df's for each sheet

# when on pc
data_test_cup = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_cup_semdis.csv")
data_test_key = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_key_semdis.csv")
data_test_rope = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_rope_semdis.csv")
data_test_brick = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_brick_semdis.csv")
data_test_chair = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_chair_semdis.csv")
data_test_pencil = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_pencil_semdis.csv")
data_test_shoe = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_shoe_semdis.csv")

# when on mac
# data_test_cup = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_cup_semdis.csv.xlsx")
# data_test_key = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_key_semdis.csv.xlsx")
# data_test_rope = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_rope_semdis.csv.xlsx")
# data_test_brick = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_brick_semdis.csv.xlsx")
# data_test_chair = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_chair_semdis.csv.xlsx")
# data_test_pencil = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_pencil_semdis.csv.xlsx")
# data_test_shoe = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_shoe_semdis.csv.xlsx")

### Preprocessing

In [4]:
# nltk corpus stop words
stopwords_nltk = stopwords.words('english')
# spacy stop words
stopwords_spacy = STOP_WORDS

In [5]:
stopwords_edited = list(stopwords_spacy)
stopwords_edited.append("thing")
stopwords_edited.append("use")
stopwords_edited.append("things")

In [6]:
# method to clean the responses
def process_text(text, stopwords_list, remove_sw, join_list):
    # tokenize text, lemmanize words, removing punctuation, remove stop words, lowercase all words

    # hardcorded for special situations
    text = re.sub("wedging","wedge", text)
    text = re.sub("exersizing","exercising", text)
    text = re.sub("thrown","throw", text)
    
    text = re.sub("/|-"," ", text)
    text = text.translate(str.maketrans('','',string.punctuation))
    tokens = word_tokenize(text)

    tokens = [w.lower() for w in tokens]
    
    if remove_sw:
        tokens = [word for word in tokens if word not in stopwords_list]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
#         stemmer = PorterStemmer()
#         tokens = [stemmer.stem(t) for t in tokens]

    if join_list:
        tokens = ' '.join(tokens)
 
    return tokens

### General Functions

In [7]:
# method to get a list of participants
def get_id_list(df):
    id_list = df['id'].unique()
    id_list = sorted(id_list)
    return id_list

In [8]:
# method to add a new column
# new column are cleaned responses
def get_cleaned_responses(df, stopwords_list, remove_sw, join_list):
    # id_df = df[df.id == id]
    df_processed = df.copy(deep=True)
    responses = df['response'].tolist()

    # make list of processed responses
    for response in range(len(responses)):
        responses[response] = process_text(responses[response], stopwords_list, remove_sw, join_list)

    # add list as column in df
    df_processed['response_processed'] = responses

    return df_processed

## Originality Algo 1
### Term Frequency Only

In [9]:
def get_tf_dict(responses):
    cv = CountVectorizer()   
    cv_fit = cv.fit_transform(responses)    
    word_list = cv.get_feature_names()

    # [0] here to get a 1d-array for iteration by the zip function 
    count_list = np.asarray(cv_fit.sum(axis=0))[0]

    tf_dict = dict(zip(word_list, count_list))
    
    total_words = 0
    for key in tf_dict:
        total_words = total_words + tf_dict[key]
        
    for key in tf_dict:
        tf_dict[key] = np.log(1 + tf_dict[key]/total_words)
        
    tf_dict = dict(sorted(tf_dict.items(), key=lambda item: item[1], reverse=True))
                
    return tf_dict

In [10]:
def get_tf_sum(tf_dict, response):
    tf_sum = 0
    for term in range(len(response)):
        tf_sum = tf_sum + tf_dict[response[term]]
        
    if tf_sum == 0:
        tf_sum = np.nan
        
    return tf_sum

In [11]:
def get_tf_sum_list(tf_dict, responses):
    tf_sum_list = []
    
    for response in responses:
        tf_sum_list.append(get_tf_sum(tf_dict, response))
            
    return tf_sum_list

In [12]:
def get_originality_tf_only(df, stopwords_list, remove_sw, join_list):
    originality_rating_df = get_cleaned_responses(df, stopwords_list, remove_sw, join_list)
    responses = originality_rating_df['response_processed'].tolist()
    responses_tokenized = [item for sublist in responses for item in sublist]

    tf_dict = get_tf_dict(responses_tokenized)
    
    tf_sum_list = get_tf_sum_list(tf_dict, responses)

    originality_rating_df['tf_sum'] = tf_sum_list
        
    return originality_rating_df

In [13]:
get_originality_tf_only(data_test_brick, stopwords_edited, True, False)

,id,response,response_nofill,item,item_nofill,SemDis_cbowukwacsubtitle_nf_m,response_processed,tf_sum
0,1476,as a foundation,foundation,Brick,brick,0.77417,[foundation],0.011834
1,1476,as a weapon,weapon,Brick,brick,0.85011,[weapon],0.068993
2,1476,as a bowl,bowl,Brick,brick,0.82926,[bowl],0.011834
3,1718,build a house,build house,Brick,brick,0.93854,"[build, house]",0.159602
4,1718,break a window,break window,Brick,brick,0.96992,"[break, window]",0.070051
5,1718,line a fireplace,line fireplace,Brick,brick,0.99511,"[line, fireplace]",0.035365
6,1718,line a road,line road,Brick,brick,0.99395,"[line, road]",0.047061
7,1691,build house,build house,Brick,brick,0.93854,"[build, house]",0.159602
8,1691,break window,break window,Brick,brick,0.96992,"[break, window]",0.070051
9,1691,weight for workout,weight workout,Brick,brick,1.05438,"[weight, workout]",0.069654


## Originality Algo 2
### tf-idf scikit-learn
### participant as document

In [14]:
def tfidf_scikit_learn(cleaned_responses):
    tfIdfVectorizer=TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(cleaned_responses)
    df = pd.DataFrame(tfIdf.toarray(), columns=tfIdfVectorizer.get_feature_names())
#     df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
#     df = df.sort_values('TF-IDF', ascending=False)
    display(df)
    
#     feature_names = tfIdfVectorizer.get_feature_names()
#     for col in tfIdf.nonzero()[1]:
#         print (feature_names[col], ' - ', tfIdf[0, col])

    print(tfIdf)

In [15]:
def get_originality_tfidf_scikit_learn(df, stopwords_list, remove_sw, join_list):
    originality_rating_df = get_cleaned_responses(df, stopwords_list, remove_sw, join_list)
    responses = []
    
    id_list = get_id_list(df)
    
    for participant in id_list:
        temp_df = originality_rating_df.loc[originality_rating_df['id'] == participant]
        temp_list = temp_df['response_processed'].tolist()
        temp_list = ' '.join(temp_list)
        responses.append(temp_list)
        
    print(responses)
    
    tfidf_scikit_learn(responses)   

In [16]:
get_originality_tfidf_scikit_learn(data_test_brick, stopwords_edited, True, True)

['foundation weapon bowl', 'house fire pit weapon break sidewalk paint', 'build hit people paperweight', 'build house break window weight workout weight holding wind karate', 'build house paperweight break prop martial art step stool', 'build house break window line fireplace line road', 'throw smash paper weight weight lifting', 'door stop weapon canvas', 'build weapon throw', 'build throw demolish paperweight', 'build weapon paperweight anchor small boat combination rope obviously', 'build house hold open door road carve name cooking', 'build throw weapon weight hard surface like anvil']


,anchor,anvil,art,boat,bowl,break,build,canvas,carve,combination,...,step,stool,stop,surface,throw,weapon,weight,wind,window,workout
0,0.000000,0.000000,0.000000,0.000000,0.655075,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.376501,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.302181,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.252086,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.277098,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.241237,0.158851,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.535518,0.350145,0.301952,0.350145
4,0.000000,0.000000,0.390781,0.000000,0.000000,0.269233,0.177286,0.000000,0.000000,0.000000,...,0.390781,0.390781,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.250556,0.164987,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.313617,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.285736,0.000000,0.634303,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.570358,0.000000,0.000000,...,0.000000,0.000000,0.570358,0.000000,0.000000,0.327811,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.451236,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.685265,0.571662,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.309031,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.469306,0.000000,0.000000,0.000000,0.000000,0.000000


  (0, 4)	0.655075136305677
  (0, 45)	0.376501170766038
  (0, 15)	0.655075136305677
  (1, 29)	0.43860410751211154
  (1, 37)	0.43860410751211154
  (1, 5)	0.3021814542943412
  (1, 33)	0.43860410751211154
  (1, 13)	0.43860410751211154
  (1, 20)	0.2750363685138143
  (1, 45)	0.2520855255053467
  (2, 31)	0.4208122865191767
  (2, 32)	0.6107919421788647
  (2, 17)	0.6107919421788647
  (2, 6)	0.2770982249189687
  (3, 21)	0.35014528605914114
  (3, 47)	0.35014528605914114
  (3, 19)	0.35014528605914114
  (3, 49)	0.35014528605914114
  (3, 46)	0.535518256916423
  (3, 48)	0.301952473284094
  (3, 6)	0.15885055209572455
  (3, 5)	0.2412367096966542
  (3, 20)	0.21956631568316437
  (4, 41)	0.39078087728443794
  (4, 40)	0.39078087728443794
  :	:
  (10, 36)	0.37767278047510044
  (10, 9)	0.37767278047510044
  (10, 3)	0.37767278047510044
  (10, 38)	0.37767278047510044
  (10, 0)	0.37767278047510044
  (10, 31)	0.2602021004744053
  (10, 6)	0.17133896150714967
  (10, 45)	0.2170655488731421
  (11, 10)	0.375653865342

## Originality Algo 3
### tf-idf scikit-learn + clustering
### participant as document


### Potential for Flexibility Algo

In [17]:
def tfidf_scikit_learn_clustering(num_clusters, cleaned_responses):
    tfidf_vectorizer=TfidfVectorizer(use_idf=True)
    tfidf = tfidf_vectorizer.fit_transform(cleaned_responses)
#     df = pd.DataFrame(tfIdf.toarray(), columns=tfIdfVectorizer.get_feature_names())
#     df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
#     df = df.sort_values('TF-IDF', ascending=False)
#     print (df)
    
    number_of_clusters = num_clusters
    kmeans = KMeans(n_clusters = num_clusters).fit(tfidf)
    
    results = pd.DataFrame()
    results['text'] = cleaned_responses
    results['category'] = kmeans.labels_
    
    results_dict = {k: g["text"].tolist() for k,g in results.groupby("category")}
    
    test = pd.DataFrame(list(results_dict.items()),columns = ['category','responses']) 
    display(test)

In [18]:
def get_originality_tfidf_scikit_learn_clustering(df, stopwords_list, num_clusters, remove_sw, join_list):
    originality_rating_df = get_cleaned_responses(df, stopwords_list, remove_sw, join_list)
    responses_split = originality_rating_df['response_processed'].tolist()
    responses = []
    
    id_list = get_id_list(df)
    
    for participant in id_list:
        temp_df = originality_rating_df.loc[originality_rating_df['id'] == participant]
        temp_list = temp_df['response_processed'].tolist()
        temp_list = ' '.join(temp_list)
        responses.append(temp_list)
            
    tfidf_scikit_learn_clustering(num_clusters, responses_split)   

In [19]:
get_originality_tfidf_scikit_learn_clustering(data_test_brick, stopwords_edited, 15, True, True)

,category,responses
0,0,"[build house, build house, house, build house,..."
1,1,"[paperweight, paperweight, paperweight, paperw..."
2,2,"[weapon, weapon, throw weapon, weapon, weapon,..."
3,3,"[line fireplace, karate, paint, step stool, ha..."
4,4,"[throw demolish, throw, throw]"
5,5,[cooking]
6,6,"[break window, break window, break, break prop..."
7,7,[hit people]
8,8,[bowl]
9,9,"[build, build, build, build, build]"


## Originality Algo 4
### Counter Vectorizer + clustering

In [38]:
def get_counts_vector(num_clusters, responses):
    # initialize CountVectorizer object
    count_vectorizer = CountVectorizer()
    # vectorize the phrases
    word_count = count_vectorizer.fit_transform(responses)
    
    # elbow method to visualize and find out how many clusters to use
#     visualizer = KElbowVisualizer(KMeans(), k=(10,35), timings=False)
#     visualizer.fit(word_count.toarray())       
#     visualizer.show()

    # nltk kmeans cosine distance implementation
    number_of_clusters = num_clusters
    kmeans = KMeansClusterer(number_of_clusters, repeats=25, distance=nltk.cluster.util.cosine_distance, avoid_empty_clusters=True)
    assigned_clusters = kmeans.cluster(word_count.toarray(), assign_clusters=True)

    # scikit-learn euclidean distance implementation
#     kmeans = KMeans(n_clusters = num_clusters).fit(word_count)
        
    # cluster results scikit-learn
    results = pd.DataFrame()
    results['text'] = responses
#     results['category'] = kmeans.labels_
    results['category'] = assigned_clusters
    
    # create dictionary to organize the clusters with their respective phrases
    results_dict = {k: g["text"].tolist() for k,g in results.groupby("category")}
    
    # df of the clusters and the 
    clusters_df = pd.DataFrame(list(results_dict.items()),columns = ['category','responses']) 
    
    return clusters_df

In [39]:
def get_clustered_originality_score(originality_rating_df, num_clusters, column):
    # get cluster df
    clusters_df = get_counts_vector(num_clusters, originality_rating_df['response_processed'].tolist())

    # create dictionary out of cluster df
    clusters = dict(zip(clusters_df.category, clusters_df.responses))
        
    # initialize empty dictionary to store the score for a category
    clusters_scores = dict.fromkeys(clusters)
    
    # initialize empty dictionary to store 
    # will store 0 or 1, 1 if only 1 response in cluster
    clusters_rarity = dict.fromkeys(clusters)
    
    # get the average cosine distance for a cluster
    for key in clusters:
        score = 1.0
        score = score - (len(clusters[key])/len(originality_rating_df.index))
        clusters_scores[key] = score
        if (len(clusters[key]) == 1):
            clusters_rarity[key] = 1
        else:
            clusters_rarity[key] = 0
        
    # create dictionary to store a phrase and its new originality score 
    # new score is the average of the responses in one cluster
    phrase_scores_dict = {}
    # create dictionary to store a phrase and its rarity 
    # will be 0 or 1, 1 if only response in that cluster
    phrase_rarity_dict = {}
    for key in clusters:
        for phrase in clusters[key]:
            phrase_scores_dict[phrase] = clusters_scores[key]
            phrase_rarity_dict[phrase] = clusters_rarity[key]
            
    # make a list that matches the one in the current dataframe
    # return list to be added to dataframe
    df_phrases_scores_list = [] 
    # make a list that matches the one in the current dataframe
    # return list to be added to dataframe
    df_phrases_rarity_list = []
    for phrase in originality_rating_df['response_processed'].tolist():
        df_phrases_scores_list.append(phrase_scores_dict[phrase])
        df_phrases_rarity_list.append(phrase_rarity_dict[phrase])
    
    # comment out to see the clusters df
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(clusters_df)
    
    print(clusters_scores)
                
    return (df_phrases_scores_list, df_phrases_rarity_list)

In [40]:
def get_originality_score_df(originality_df):
    # get id list and create empty dict
    id_list = get_id_list(originality_df)
    participants_originality = {k: 0 for k in id_list}
    # add the number of unique responses aka clusters with only 1 response
    for participant in id_list:
        id_df = originality_df[originality_df.id == participant]
        participants_originality[participant] = id_df['originality'].sum()
        
    return pd.DataFrame(participants_originality.items(), columns=['id', 'originality'])

In [41]:
def get_originality_count_vectorizer(df, stopwords_list, num_clusters, remove_sw, join_list):
    originality_rating_df = get_cleaned_responses(df, stopwords_list, remove_sw, join_list)
    responses_split = originality_rating_df['response_processed'].tolist()
    originality_rating_df = originality_rating_df[originality_rating_df.astype(str)['response_processed'] != '']
    responses = []
    
#     responses_tokenized = [item for sublist in responses for item in sublist]

    id_list = get_id_list(df)

    for participant in id_list:
        temp_df = originality_rating_df.loc[originality_rating_df['id'] == participant]
        temp_list = temp_df['response_processed'].tolist()
        temp_list = ' '.join(temp_list)
        responses.append(temp_list)
            
    originality_results = get_clustered_originality_score(originality_rating_df, num_clusters, responses_split)
    # add frequency column to results df
    originality_rating_df['cluster_freq'] = originality_results[0]
    # add 1/0 column to show if that phrase was by itself in a cluster
    originality_rating_df['originality'] = originality_results[1]
    
    # get the original phrases counts df
    originality_scores = get_originality_score_df(originality_rating_df)

    return (originality_rating_df, originality_scores)

In [42]:
originality_results = get_originality_count_vectorizer(data_official_rope, stopwords_edited, 80, True, True)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(originality_results)

    category                                          responses
0          0               [whip, whip, whip, whip, whip, whip]
1          1                                            [sport]
2          2                                       [kill, kill]
3          3                                       [jump, jump]
4          4  [tie, tie, tie, tie, tie, tie, tie, tie, tie, ...
5          5                                    [hit, hit, hit]
6          7                                         [strangle]
7         10                                             [walk]
8         11  [climb, climb, climb, climb, climb, climb, cli...
9         12                   [weapon, weapon, weapon, weapon]
10        13                           [strangling, strangling]
11        14                                         [exercise]
12        15  [lasso, lasso, lasso, lasso, lasso, lasso, las...
13        16   [knot, knot, knot, knot, knot, knot, knot, knot]
14        17                            

In [43]:
originality_results[1]

,id,originality
0,1087,0
1,1093,0
2,1094,0
3,1102,0
4,1104,0
...,...,...
85,1603,0
86,1610,0
87,1614,0
88,1621,0


## Algo Results with Official Data

In [26]:
# # when on pc
# data_official_cup = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Cup')
# data_official_key = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Key')
# data_official_rope = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Rope')
# data_official_brick = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Brick')
# data_official_chair = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Chair')
# data_official_pencil = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Pencil')
# data_official_shoe = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Shoe')
# data_official_box = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Box')

# # when on mac
# # data_official_cup = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Cup')
# # data_official_key = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Key')
# # data_official_rope = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Rope')
# # data_official_brick = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Brick')
# # data_official_chair = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Chair')
# # data_official_pencil = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Pencil')
# # data_official_shoe = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Shoe')
# # data_official_box = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_official.xlsx", sheet_name='Box')

In [29]:
# individual df's for each sheet

# when on pc
data_official_cup = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_cup_semdis.csv")
data_official_key = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_key_semdis.csv")
data_official_rope = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_rope_semdis.csv")
data_official_brick = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_brick_semdis.csv")
data_official_chair = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_chair_semdis.csv")
data_official_pencil = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_pencil_semdis.csv")
data_official_shoe = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_shoe_semdis.csv")
data_official_box = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_box_semdis.csv")

# when on mac
# data_official_cup = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_cup_semdis.csv")
# data_official_key = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_key_semdis.csv")
# data_official_rope = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_rope_semdis.csv")
# data_official_brick = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_brick_semdis.csv")
# data_official_chair = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_chair_semdis.csv")
# data_official_pencil = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_pencil_semdis.csv")
# data_official_shoe = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_shoe_semdis.csv")
# data_official_box = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_box_semdis.csv")

In [44]:
prompts_list = ['box', 'brick', 'chair', 'cup', 'key', 'pencil', 'rope', 'shoe']
data_list = [data_official_box, data_official_brick, data_official_chair, data_official_cup, data_official_key, data_official_pencil, data_official_rope, data_official_shoe]

In [45]:
def get_originality_scores(data_list, num_clusters):
    results_list = []
    for i in range(len(data_list)):
        # get originality results for each dataset
        results = get_originality_count_vectorizer(data_list[i], stopwords_edited, num_clusters, True, True)
        # add originality results to a list
        results_list.append(results)
        
    # return list
    return results_list

In [46]:
originality_results_list = get_originality_scores(data_list, 80)

    category                                          responses
0          0                [chair, chair, chair, chair, chair]
1          1                               [organize, organize]
2          2                                         [dragging]
3          3                                           [office]
4          4                                             [sail]
5          5   [fort, fort, fort, fort, fort, fort, fort, fort]
6          6                                       [sled, sled]
7          8                           [shoe, shoe, shoe, shoe]
8          9                               [standing, standing]
9         10  [stand, stand, stand, stand, stand, stand, sta...
10        15  [store, store, store, store, store, store, sto...
11        16                                       [protecting]
12        17                                     [plate, plate]
13        19      [present, present, present, present, present]
14        20                       [stoo

    category                                          responses
0          0                                    [hit, hit, hit]
1          1                                             [push]
2          3                [throw, throw, throw, throw, throw]
3          4                                           [dining]
4          5  [sitting, sitting, sitting, sitting, sitting, ...
5          6  [sleeping, sleeping, sleeping, sleeping, sleep...
6          7                                            [guest]
7          8                                          [hitting]
8         10  [sit, sit, sit, sit, sit, sit, sit, sit, sit, ...
9         11                        [storage, storage, storage]
10        14                                 [support, support]
11        15                                         [weighing]
12        17                [sleep, sleep, sleep, sleep, sleep]
13        20         [table, table, table, table, table, table]
14        21  [weapon, weapon, weapon, w

    category                                          responses
0          0                       [chain, chain, chain, chain]
1          1                                       [scratching]
2          2                                          [fashion]
3          3  [jewelry, jewelry, jewelry, jewelry, jewelry, ...
4          4                                       [kill, kill]
5          6  [necklace, necklace, necklace, necklace, neckl...
6          7  [weapon, weapon, weapon, weapon, weapon, weapo...
7          8                                       [persuasion]
8         10                                             [door]
9         11  [cut, cut, cut half, cut, cut, cut, cut, cut, ...
10        12  [decoration, decoration, decoration, decoratio...
11        13                       [house, unlock house, house]
12        14  [unlocking, unlocking, unlocking, unlocking do...
13        15  [separate stuck, hiding, discovering, music, j...
14        16                            

    category                                          responses
0          0                                            [repel]
1          1                                        [loosening]
2          2                                          [shaking]
3          3                                          [weaving]
4          4                              [leash, leash, leash]
5          5  [tie, tie, tie, tie, tie, tie, tie, tie, tie, ...
6          6  [climb, climb, climb, climb, climb, climb, cli...
7          7   [knot, knot, knot, knot, knot, knot, knot, knot]
8          8                                          [boarder]
9         11                                         [painting]
10        12  [swing, swing, swing, swing, swing, swing, swi...
11        13               [whip, whip, whip, whip, whip, whip]
12        15  [lasso, lasso, lasso, lasso, lasso, lasso, las...
13        18                                         [sailboat]
14        23                            

In [47]:
originality_results_list[0][1]

,id,originality
0,1087,0
1,1093,0
2,1094,1
3,1102,1
4,1104,0
...,...,...
84,1599,0
85,1603,0
86,1610,0
87,1614,0


## Write Originality Results into Excel

In [48]:
# write originality results into excel sheet
def write_originality_results(results):
    writer = pd.ExcelWriter('originality_methods_results_071321.xlsx', engine='xlsxwriter')
    workbook = writer.book
    
    for i in range(len(prompts_list)):
        worksheet = workbook.add_worksheet(prompts_list[i])
        writer.sheets[prompts_list[i]] = worksheet
        row = 0
        worksheet.write_string(row, 0, prompts_list[i])
        results[i][0].to_excel(writer, sheet_name = prompts_list[i], startrow=row + 1, startcol=0, index = False)
#         row = row + len(results[i][0].index) + 3
        worksheet.write_string(row, len(results[i][0].columns) + 3, 'participants originality score')
        results[i][1].to_excel(writer, sheet_name = prompts_list[i], startrow=row + 1, startcol= len(results[i][0].columns) + 3, index = False)
#         row = row + len(results[i][1].index) + 3
            
    writer.save()

In [49]:
write_originality_results(originality_results_list)

Algo Design Brainstorming:
* Algo Design w/o scikit-learn
    * asdlfjakl;fjka
    * asdfasf

* Algo Design w/ scikit-learn

To Do List
- [x] write basic algorithm, sum of term frequency
    - frequency based on all phrases
- [x] write tfidf clustering method 
    - repurpose for flexibility
- [x] write counter vectorizer clustering method
- [x] write method to score responses by frequency
    - number of responses in a cluster/number of total responses subtracted from 1
    - subtract from 1 to get the reverse mapping
- [x] experiment with a greater cluster size
- [x] update stop words list to include "use" and "thing"
- [ ] figure out how many times to run kmeans
    - cross validation
    - then averaging the results of all the iterations
